In [ ]:
import numpy
import matplotlib.pyplot as plt
import pandas as pd
from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, \
    TimeSeriesResampler

In [ ]:
df = pd.read_csv('C:\\Users\\alexandre.lucas\\Desktop\\INESCTEC\Asprela + Sustentavel - Living Labs Porto\\DATA_SET_DESAGREGADO_v.2.0.csv', sep=";")
df.head()

In [ ]:
df1=df.drop("Horas", axis=1)
df1=df1.drop("HSJ_IPO_Metro_Cantina", axis=1) #Outline
df1=df1.drop("PTD_54", axis=1) #Coluna com zeros
#df1=df.drop("FEUP", axis=1) #Comportamento atipico que cria o seu proprio cluster. Referenciá-lo à parte
df1=df1.transpose()

In [ ]:
df1.head()

In [ ]:
df1.shape
X_train=df1.iloc[0:87, 0:168]
#X_train=df1
X_train

In [ ]:
from sklearn import preprocessing
x_scaled=preprocessing.scale(X_train)
X_train=x_scaled
X_train

In [ ]:
#X_train = TimeSeriesScalerMeanVariance().fit_transform(X_train)
#X_train=X_train.iloc[1:300] #de 1 a x linhas todas a colunas
#np.reshape(a, (2, 3)) # C-like index ordering
#array([[0, 1, 2],
#       [3, 4, 5]])

#X_train = TimeSeriesResampler(sz=40).fit_transform(X_train) # Make time series shorter

X_train.shape

In [ ]:
seed = 0
numpy.random.seed(seed)
sz = X_train.shape[1]

In [ ]:
# Soft-DTW-k-means
kmeans_kwargs = {
                 "metric":"softdtw",
                "metric_params":{"gamma": .01},
                "verbose":True,
                "random_state":seed,}

# A list holds the SSE values for each k
sse = []
for k in range(1, 6):  
    sdtw_km = TimeSeriesKMeans(n_clusters=k, **kmeans_kwargs)
    y_pred = sdtw_km.fit_predict(X_train)
    sse.append(sdtw_km.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 6), sse)
plt.xticks(range(1, 6))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
# DTW-k-means
print("DTW k-means")
dtw_km = TimeSeriesKMeans(n_clusters=2,
                          n_init=2,
                          metric="dtw",
                          verbose=True,
                          max_iter_barycenter=10,
                          random_state=seed)
y_pred = dtw_km.fit_predict(X_train)

In [ ]:
#plt.figure()
plt.figure(figsize=(20,15))
plt.subplots(figsize=(12, 4))
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}

plt.rc('font', **font)       

for yi in range(2):
    plt.subplot(1, 2, 1 + yi)
    for xx in X_train[y_pred == yi]:
        plt.plot(xx.ravel(), "g-", alpha=.2)
    plt.plot(dtw_km.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
#    plt.ylim(-1.5, 8)
    plt.text(0.45, 0.9,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
#    if yi == 1:
#        plt.title("DTW $k$-means")
        

plt.tight_layout()
plt.show()

In [ ]:
dtw_km.labels_